In [ ]:

from datetime import date
import os

from strava.data.strava_requests import (
    create_stream_df, 
    get_activity_stream_by_date, 
    activity_id_by_date,
)
from strava.data.cache import Cache
from strava.data.segment_access import (
    create_segment_catalog,
    segment_id_from_activity_date_and_index,
)
from strava.plotting.strava_stream_plots import (
    heartrate_with_altitude,
    velocity_with_altitude,
    heartrate_zones,
    all_streams,
    combine_plots_vertical,
)
from strava.plotting.strava_annual_plots import annual_plot
from strava.plotting.segment_plots import segment_effort_graph
from generate_plots import get_clean_activities

%load_ext autoreload
%autoreload 2

# General Setup

In [2]:
# Activities Cache
cache = Cache()
activity_cache = Cache(dir='/Users/lucasnieuwenhout/Documents/Programming/Python/Projects/StravaPlotting/activity_cache/')

In [3]:
# Retrieve the list of activities
activities = get_clean_activities()

Existing access token is valid.
Beginning request of activities...
-> Received page 1 with 200 activities.
-> Received page 2 with 200 activities.
-> Received page 3 with 200 activities.
-> Received page 4 with 75 activities.
-> Received page 5 with 0 activities.
...retrieved total of 675 activities.


/Users/lucasnieuwenhout/Documents/Programming/Python/Projects/StravaPlotting/src/generate_plots.py:150: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.


# Create Summary Plot

### Retrieve Stream

In [4]:
ride_date = date.today()
ride_date = date(2025, 5, 12)
ride_date = activities.sort_values('id', ascending=False).iloc[0].start_date_local.date()

In [5]:
stream = create_stream_df(
    get_activity_stream_by_date(activities=activities, 
                                date=ride_date, 
                                cache=cache), 
    activity_id_by_date(activities=activities, 
                        date=ride_date)
    )

### Create Plots

In [6]:
heartrate_plot = heartrate_with_altitude(stream=stream)
velocity_plot = velocity_with_altitude(stream=stream)
(heartrate_zone_plot, zone_plot) = heartrate_zones(stream=stream, heartrate_max=191)
summary = all_streams(stream, alpha=0.1)

### Save Plots

In [7]:
plot_height = 24
plot_width  = 48
transparent_background = False
heartrate_plot.save(filename="../plots/plot_components/heartrate_plot.png", format="png", height=plot_height, width=plot_width, limitsize=False, transparent=transparent_background)
velocity_plot.save(filename="../plots/plot_components/velocity_plot.png", format="png", height=plot_height, width=plot_width, limitsize=False, transparent=transparent_background)
heartrate_zone_plot.save(filename="../plots/plot_components/heartrate_zone_plot.png", format="png", height=plot_height, width=plot_width, limitsize=False, transparent=transparent_background)
zone_plot.save(filename="../plots/plot_components/zone_plot.png", format="png", height=4, width=48, limitsize=False, transparent=transparent_background)
summary.save(filename="../plots/plot_components/stream_summary.png", format="png", height=plot_height, width=plot_width, limitsize=False, transparent=transparent_background)

id = stream.iloc[0]["id"]
with open(f"../plots/summary_{ride_date.isoformat()}_{id}.png", "wb") as fp:
    combine_plots_vertical(["../plots/plot_components/velocity_plot.png", "../plots/plot_components/heartrate_plot.png", "../plots/plot_components/heartrate_zone_plot.png", "../plots/plot_components/zone_plot.png"]).save(fp=fp, format="png")

/Users/lucasnieuwenhout/miniforge3/envs/env_general/lib/python3.10/site-packages/plotnine/layer.py:372: PlotnineWarning: geom_rect : Removed 1 rows containing missing values.
/Users/lucasnieuwenhout/miniforge3/envs/env_general/lib/python3.10/site-packages/plotnine/geoms/geom_path.py:100: PlotnineWarning: geom_path: Removed 9 rows containing missing values.


# Create Annual Plots

In [8]:
subjects = ["heartrate", "speed", "distance", "elevation", "annual_time"]
plots = [annual_plot(activities=activities, subject=x) for x in subjects]

for i in range(len(subjects)):
    filename = f"../plots/annual/{subjects[i]}.png"
    plots[i].save(filename=filename, format="png", height=9, width=30, transparent=transparent_background, limitsize=False)

/Users/lucasnieuwenhout/miniforge3/envs/env_general/lib/python3.10/site-packages/plotnine/layer.py:372: PlotnineWarning: geom_label : Removed 934 rows containing missing values.
/Users/lucasnieuwenhout/miniforge3/envs/env_general/lib/python3.10/site-packages/plotnine/layer.py:372: PlotnineWarning: geom_label : Removed 934 rows containing missing values.
/Users/lucasnieuwenhout/miniforge3/envs/env_general/lib/python3.10/site-packages/plotnine/scales/scale_manual.py:45: PlotnineWarning: The palette of scale_color_manual can return a maximum of 3 values. 6 were requested from it.


# Create Relevant Segments

### Retrieve Segments

In [9]:
# bc_activities = activities[activities.start_latlng.apply(lambda x: float(x[0])) < 50]
segment_catalog = create_segment_catalog(activities, activity_cache)

### Plot Segments

In [10]:
segments_path = r'/Users/lucasnieuwenhout/Documents/Programming/Python/Projects/StravaPlotting/plots/segments/'
latest_segments_path = r'/Users/lucasnieuwenhout/Documents/Programming/Python/Projects/StravaPlotting/plots/latest_segments/'
segment_save_paths = [segments_path, latest_segments_path]

In [11]:
ride_segments = segment_id_from_activity_date_and_index(activities=activities,
                                                        activity_cache=activity_cache,
                                                        activity_date=ride_date,
                                                        entire_list=True)
os.system(f'rm {latest_segments_path}*')
for segment_id in ride_segments:
    segment_effort_graph(segment_catalog=segment_catalog,
                        segment_id=segment_id,
                        segment_plot_path=segment_save_paths,
                        previous_x=8)